# **Setup GPU**

In [1]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# **Importing the required packages**

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np
from numpy import random
from pickle import load
from numpy import array
import os
import re
import pandas as pd
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras import optimizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.metrics import RootMeanSquaredError
import matplotlib.pyplot as plt
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import KFold

[nltk_data] Downloading package stopwords to /home/coss09/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Building embedding for the words**

In [3]:
# embedding bin file
embed_file = "10k-sample/sim.expand.200d.vec"

#Define Hyper parameters
max_inp_len = 20000
# the dimension of vectors to be used
embed_dim = 200
rounding = 6
# filter sizes of the different conv layers 
filter_sizes = [3,4,5]
num_filters = 1
pool_size = 199
# dropout probability
drop = 0.5
batch_size = 50
learning_rate = 0.001
epochs = 30

In [4]:
#define embedding dictionary and embed matrix for the vocabulary
embeddings_dic = dict()
f = open(embed_file,encoding='utf8')
with open(embed_file, 'r', encoding='utf-8') as e_file:
  for line in e_file:
    splitlines = line.split()
    word = splitlines[0].strip()
    coefs = np.asarray(splitlines[1:], dtype='float32')
    embeddings_dic[word] = coefs

print("length of embedding dictionary",len(embeddings_dic))

length of embedding dictionary 70429


In [5]:
vocabulary_size = len(embeddings_dic.keys())
embed_token = Tokenizer()
embed_token.fit_on_texts(embeddings_dic.keys())
embedding_matrix = np.zeros((vocabulary_size, embed_dim))
for word, index in embed_token.word_index.items():
    embedding_matrix[index] = embeddings_dic.get(word)
print("embedding_matrix dimension",len(embedding_matrix),len(embedding_matrix[0]))
print("no of token in the tokenizer",len(embed_token.word_index) + 1)

embedding_matrix dimension 70429 200
no of token in the tokenizer 70429


# **Functions to pre process input and output**

In [6]:
#function to pre process the document
def process_doc(path_file,embed_token) :

  #tokenizing the words 
  with open(path_file,'r', encoding='utf-8') as tok_file :
    file_words = list(tok_file)[0].split()
    
  #removing the stop words
  stop_words = set(stopwords.words('english'))
  filtered_words = []  
  for word in file_words: 
      if word not in stop_words and word.isalpha(): 
          filtered_words.append(word)

  # applying stemming using PorterStemmer

  p_stemmer = PorterStemmer()
  stem_words=[]
  for word in filtered_words:
    stem_words.append(p_stemmer.stem(word))
    
  #tokenizing the words using the embed token
  tokens=[]
  for word in stem_words:
    try:
      tokens.append(embed_token.word_index[word])
    except:
      tokens.append(1)

  if len(tokens) < max_inp_len:
    tokens.extend([0]*(max_inp_len-len(tokens)))
  else:
    tokens = tokens[:max_inp_len]
    
  return np.array(tokens)

In [7]:
#output dataset
def output_data(company_id, out_path_file):
  with open(out_path_file,'r', encoding='utf-8') as out_file :
    for line in out_file.readlines():
      if company_id == line.split()[1]:
        return line.split()[0]
  return None

In [8]:
def pre_processing(meta_file,output_file):

  with open(meta_file,'r', encoding='utf-8') as m_file :
    
    year = meta_file.split('/')[2].split('.')[0]
    dir_path = os.path.dirname(meta_file).split('/')[0] + '/all.tok/' +year+'.tok'
    data =[]

    for line in m_file.readlines():
      inp_path_file = dir_path +'/'+ line.split()[0] + '.mda'
      
      # get input tokens from the company document
      inp_tokens = process_doc(inp_path_file,embed_token)
      
      # get output value for the company
      out_values = output_data(line.split()[0],output_file)

      #insert values into the data list
      data.append({'token':inp_tokens,'value':out_values})

  return data

# **Define the model**

In [9]:
def define_model(max_inp_len,vocabulary_size,embed_dim,filter_sizes,num_filters,pool_size,drop,learning_rate):
    
    # input and embedding matrix
    inputs = Input(shape=(max_inp_len,))
    embedding = Embedding(vocabulary_size, embed_dim, weights=[embedding_matrix],trainable = False)(inputs)
    
    custom_objects={'leaky_relu': tf.nn.leaky_relu}
    
    # channel 1 convolution and local max-pooling
    convolution_1 = Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation=custom_objects['leaky_relu'])(embedding)
    pool_1 = MaxPooling1D(pool_size=pool_size)(convolution_1)
    
    # channel 2 convolution and local max-pooling
    convolution_4 = Conv1D(filters=num_filters, kernel_size=filter_sizes[1], activation=custom_objects['leaky_relu'])(embedding)
    pool_2 = MaxPooling1D(pool_size=pool_size)(convolution_4)
    
    # channel 3 convolution and local max-pooling
    convolution_5 = Conv1D(filters=num_filters, kernel_size=filter_sizes[2], activation=custom_objects['leaky_relu'])(embedding)
    pool_3 = MaxPooling1D(pool_size=pool_size)(convolution_5)
    
    # merge and dropout
    merged = concatenate([pool_1,pool_2,pool_3],axis=1)
    drop_out = Dropout(drop)(merged)
    flat = Flatten()(drop_out)
    
    # 2 fully connected layers
    dense1 = Dense(100, activation=custom_objects['leaky_relu'])(flat)
    outputs = Dense(1, activation=custom_objects['leaky_relu'])(dense1)
    model = Model(inputs=[inputs], outputs=outputs)
    
    opt = optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=opt)
    
    return model

In [10]:
# define model
model = define_model(max_inp_len,vocabulary_size,embed_dim,filter_sizes,num_filters,pool_size,drop,learning_rate)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20000)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20000, 200)   14085800    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 19998, 1)     601         embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 19997, 1)     801         embedding[0][0]                  
______________________________________________________________________________________________

# **Data extraction and Fitting the model**

In [ ]:
test_loss_all_years = []
data = []
n_splits = 5
for year in range(2008,2014):
    
    ######## extracting text and storing it in dataframes ########
#     with tf.device('/device:GPU:0'):
    data_train = pre_processing('10k-sample/all.meta/'+str(year-3)+'.meta.txt','10k-sample/all.logfama/'+str(year-3)+'.logfama.txt')
    data_train.extend(pre_processing('10k-sample/all.meta/'+str(year-2)+'.meta.txt','10k-sample/all.logfama/'+str(year-2)+'.logfama.txt'))
    data_train.extend(pre_processing('10k-sample/all.meta/'+str(year-1)+'.meta.txt','10k-sample/all.logfama/'+str(year-1)+'.logfama.txt'))
    train_df = pd.DataFrame(data_train,columns=['token','value'])

    data_test = pre_processing('10k-sample/all.meta/'+str(year)+'.meta.txt','10k-sample/all.logfama/'+str(year)+'.logfama.txt')
    test_df = pd.DataFrame(data_test,columns=['token','value'])
    data.append({'year':year,'train_df_length':len(data_train),'test_df_length':len(data_test)})
    
    ######## reshapping data to required format ########
    CNN_train_input = train_df.token.values
    CNN_train_output = [ float(x) for x in train_df.value.values ]
    CNN_test_input = test_df.token.values
    CNN_test_output = [ float(x) for x in test_df.value.values ]
    CNN_train_output = np.array(CNN_train_output).reshape(len(CNN_train_output),1)
    CNN_test_output = np.array(CNN_test_output).reshape(len(CNN_test_output),1)
    
    CNN_train_input = np.stack(CNN_train_input)
    CNN_test_input = np.stack(CNN_test_input)
    
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    CNN_train_output = np.array(CNN_train_output).reshape(len(CNN_train_output),1)
    CNN_test_output = np.array(CNN_test_output).reshape(len(CNN_test_output),1)
    output = np.concatenate((CNN_train_output, CNN_test_output))
    output = scaler.fit_transform(output)
    CNN_train_output = output[:len(CNN_train_input)]
    albert_test_output = output[-len(CNN_test_input):]
    
    
    

    ######## Kfold training and saving checkpoints ########
#     with tf.device('/device:GPU:0'):
    kf = KFold(n_splits=n_splits)
    history =[]
    train_loss=[]
    vald_loss=[]
    test_loss = []
    fold = 1

    for train_index, test_index in kf.split(CNN_train_input):

        checkpoint_filepath = 'Results/CNN_results_min/CheckPoints/'+str(year)+'CNN_checkpoint'+str(fold)
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                                filepath=checkpoint_filepath,
                                                save_weights_only=False,
                                                monitor='loss',
                                                mode='min',
                                                save_best_only=True)
        train_history = model.fit(
                              CNN_train_input[train_index],
                              CNN_train_output[train_index],#output
                              epochs=epochs, #epochs
                              verbose=1,
                              callbacks=[model_checkpoint_callback]
                          )
        model_best = tf.keras.models.load_model(checkpoint_filepath)
        fold+=1
        loss_T = model_best.evaluate(CNN_train_input[train_index],CNN_train_output[train_index], verbose=0)
        loss_V = model_best.evaluate(CNN_train_input[test_index],CNN_train_output[test_index], verbose=0)
        loss_test = model_best.evaluate(CNN_test_input,CNN_test_output, verbose=0)

        train_loss.append(loss_T)
        vald_loss.append(loss_V)
        history.append(train_history)
        test_loss.append(loss_test)
            
    test_loss_all_years.append(test_loss)

Epoch 1/30
194/194 [==============================] - ETA: 0s - loss: 0.0828WARNING:tensorflow:From /home/coss09/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/resource_variable_ops.py:1813: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: Results/CNN_results_min/CheckPoints/2008CNN_checkpoint1/assets
194/194 [==============================] - 394s 2s/step - loss: 0.0828
Epoch 2/30
194/194 [==============================] - 394s 2s/step - loss: 0.0753
Epoch 3/30
194/194 [==============================] - 394s 2s/step - loss: 0.0716
Epoch 4/30
194/194 [==============================] - 394s 2s/step - loss: 0.0676
Epoch 5/30
194/194 [==============================] - 394s 2s/step - loss: 0.0637
Epoch 6/30
194/194 [==============================] - 395s

194/194 [==============================] - 419s 2s/step - loss: 0.0418
Epoch 9/30
194/194 [==============================] - 421s 2s/step - loss: 0.0416
Epoch 10/30
194/194 [==============================] - 427s 2s/step - loss: 0.0417
Epoch 11/30
194/194 [==============================] - 433s 2s/step - loss: 0.0414
Epoch 12/30
194/194 [==============================] - 435s 2s/step - loss: 0.0418
Epoch 13/30
194/194 [==============================] - 435s 2s/step - loss: 0.0411
Epoch 14/30
194/194 [==============================] - 435s 2s/step - loss: 0.0410
Epoch 15/30
194/194 [==============================] - 436s 2s/step - loss: 0.0411
Epoch 16/30
194/194 [==============================] - 436s 2s/step - loss: 0.0408
Epoch 17/30
194/194 [==============================] - 435s 2s/step - loss: 0.0410
Epoch 18/30
194/194 [==============================] - 437s 2s/step - loss: 0.0407
Epoch 19/30
194/194 [==============================] - 442s 2s/step - loss: 0.0405
Epoch 20/30
194/1

194/194 [==============================] - 500s 3s/step - loss: 0.0417
Epoch 12/30
194/194 [==============================] - 500s 3s/step - loss: 0.0415
Epoch 13/30
194/194 [==============================] - 503s 3s/step - loss: 0.0413
Epoch 14/30
194/194 [==============================] - 503s 3s/step - loss: 0.0415
Epoch 15/30
194/194 [==============================] - 505s 3s/step - loss: 0.0413
Epoch 16/30
194/194 [==============================] - 504s 3s/step - loss: 0.0412
Epoch 17/30
194/194 [==============================] - 503s 3s/step - loss: 0.0413
Epoch 18/30
194/194 [==============================] - 506s 3s/step - loss: 0.0412
Epoch 19/30
194/194 [==============================] - 507s 3s/step - loss: 0.0413
Epoch 20/30
194/194 [==============================] - 506s 3s/step - loss: 0.0411
Epoch 21/30
194/194 [==============================] - 505s 3s/step - loss: 0.0411
Epoch 22/30
194/194 [==============================] - 506s 3s/step - loss: 0.0414
Epoch 23/30
194/

Epoch 17/30
190/190 [==============================] - 432s 2s/step - loss: 0.0370
Epoch 18/30
190/190 [==============================] - 435s 2s/step - loss: 0.0367
Epoch 19/30
190/190 [==============================] - 432s 2s/step - loss: 0.0369
Epoch 20/30
190/190 [==============================] - 436s 2s/step - loss: 0.0369
Epoch 21/30
190/190 [==============================] - 445s 2s/step - loss: 0.0369
Epoch 22/30
190/190 [==============================] - 462s 2s/step - loss: 0.0367
Epoch 23/30
190/190 [==============================] - 468s 2s/step - loss: 0.0368
Epoch 24/30
190/190 [==============================] - 480s 3s/step - loss: 0.0369
Epoch 25/30
190/190 [==============================] - 483s 3s/step - loss: 0.0368
Epoch 26/30
190/190 [==============================] - 486s 3s/step - loss: 0.0368
Epoch 27/30
190/190 [==============================] - 492s 3s/step - loss: 0.0369
Epoch 28/30
190/190 [==============================] - 500s 3s/step - loss: 0.0368
Epoc

190/190 [==============================] - 483s 3s/step - loss: 0.0363
Epoch 27/30
190/190 [==============================] - 482s 3s/step - loss: 0.0366
Epoch 28/30
190/190 [==============================] - 482s 3s/step - loss: 0.0364
Epoch 29/30
190/190 [==============================] - 484s 3s/step - loss: 0.0363
Epoch 30/30
190/190 [==============================] - 492s 3s/step - loss: 0.0365
Epoch 1/30
190/190 [==============================] - 492s 3s/step - loss: 0.0360
Epoch 2/30
190/190 [==============================] - 490s 3s/step - loss: 0.0356
Epoch 3/30
190/190 [==============================] - 485s 3s/step - loss: 0.0360
Epoch 4/30
190/190 [==============================] - 439s 2s/step - loss: 0.0359
Epoch 5/30
190/190 [==============================] - 419s 2s/step - loss: 0.0360
Epoch 6/30
190/190 [==============================] - 419s 2s/step - loss: 0.0359
Epoch 7/30
190/190 [==============================] - 420s 2s/step - loss: 0.0359
Epoch 8/30
190/190 [===

...

Epoch 19/30
190/190 [==============================] - 387s 2s/step - loss: 0.0384
Epoch 20/30
190/190 [==============================] - 387s 2s/step - loss: 0.0381
Epoch 21/30
190/190 [==============================] - 388s 2s/step - loss: 0.0382
Epoch 22/30
190/190 [==============================] - 390s 2s/step - loss: 0.0380
Epoch 23/30
190/190 [==============================] - 390s 2s/step - loss: 0.0382
Epoch 24/30
190/190 [==============================] - 398s 2s/step - loss: 0.0382
Epoch 25/30
190/190 [==============================] - 404s 2s/step - loss: 0.0382
Epoch 26/30
190/190 [==============================] - 414s 2s/step - loss: 0.0379
Epoch 27/30
190/190 [==============================] - 416s 2s/step - loss: 0.0383
Epoch 28/30
190/190 [==============================] - 415s 2s/step - loss: 0.0383
Epoch 29/30
190/190 [==============================] - 415s 2s/step - loss: 0.0383
Epoch 30/30
190/190 [==============================] - 416s 2s/step - loss: 0.0381
Epoc

188/188 [==============================] - 485s 3s/step - loss: 0.0457
Epoch 7/30
188/188 [==============================] - 483s 3s/step - loss: 0.0457
Epoch 8/30
188/188 [==============================] - 483s 3s/step - loss: 0.0455
Epoch 9/30
188/188 [==============================] - 484s 3s/step - loss: 0.0455
Epoch 10/30
188/188 [==============================] - 487s 3s/step - loss: 0.0456
Epoch 11/30
188/188 [==============================] - 484s 3s/step - loss: 0.0457
Epoch 12/30
188/188 [==============================] - 485s 3s/step - loss: 0.0454
Epoch 13/30
188/188 [==============================] - 484s 3s/step - loss: 0.0455
Epoch 14/30
188/188 [==============================] - 486s 3s/step - loss: 0.0455
Epoch 15/30
188/188 [==============================] - 486s 3s/step - loss: 0.0454
Epoch 16/30
188/188 [==============================] - 485s 3s/step - loss: 0.0454
Epoch 17/30
188/188 [==============================] - 489s 3s/step - loss: 0.0454
Epoch 18/30
188/188

188/188 [==============================] - 435s 2s/step - loss: 0.0434
Epoch 16/30
188/188 [==============================] - 435s 2s/step - loss: 0.0435
Epoch 17/30
188/188 [==============================] - 434s 2s/step - loss: 0.0434
Epoch 18/30
188/188 [==============================] - 433s 2s/step - loss: 0.0434
Epoch 19/30
188/188 [==============================] - 432s 2s/step - loss: 0.0432
Epoch 20/30
188/188 [==============================] - 431s 2s/step - loss: 0.0435
Epoch 21/30
188/188 [==============================] - 435s 2s/step - loss: 0.0432
Epoch 22/30
188/188 [==============================] - 429s 2s/step - loss: 0.0436
Epoch 23/30
188/188 [==============================] - 428s 2s/step - loss: 0.0434
Epoch 24/30
188/188 [==============================] - 428s 2s/step - loss: 0.0433
Epoch 25/30
188/188 [==============================] - 429s 2s/step - loss: 0.0434
Epoch 26/30
188/188 [==============================] - 431s 2s/step - loss: 0.0433
Epoch 27/30
188/

188/188 [==============================] - 493s 3s/step - loss: 0.0417
Epoch 29/30
188/188 [==============================] - 486s 3s/step - loss: 0.0418
Epoch 30/30
188/188 [==============================] - 486s 3s/step - loss: 0.0415
Epoch 1/30
186/186 [==============================] - 493s 3s/step - loss: 0.0442
Epoch 2/30
186/186 [==============================] - 489s 3s/step - loss: 0.0437
Epoch 3/30
186/186 [==============================] - 488s 3s/step - loss: 0.0438
Epoch 4/30
186/186 [==============================] - 490s 3s/step - loss: 0.0437
Epoch 5/30
186/186 [==============================] - 463s 2s/step - loss: 0.0437
Epoch 6/30
186/186 [==============================] - 417s 2s/step - loss: 0.0435
Epoch 7/30
186/186 [==============================] - 420s 2s/step - loss: 0.0436
Epoch 8/30
186/186 [==============================] - 417s 2s/step - loss: 0.0435
Epoch 9/30
186/186 [==============================] - 412s 2s/step - loss: 0.0437
Epoch 10/30
186/186 [====

186/186 [==============================] - 482s 3s/step - loss: 0.0423
Epoch 6/30
186/186 [==============================] - 476s 3s/step - loss: 0.0422
Epoch 7/30
186/186 [==============================] - 478s 3s/step - loss: 0.0422
Epoch 8/30
186/186 [==============================] - 481s 3s/step - loss: 0.0424
Epoch 9/30
186/186 [==============================] - 484s 3s/step - loss: 0.0423
Epoch 10/30
186/186 [==============================] - 491s 3s/step - loss: 0.0421
Epoch 11/30
186/186 [==============================] - 490s 3s/step - loss: 0.0422
Epoch 12/30
186/186 [==============================] - 491s 3s/step - loss: 0.0425
Epoch 13/30
186/186 [==============================] - 411s 2s/step - loss: 0.0424
Epoch 14/30
186/186 [==============================] - 406s 2s/step - loss: 0.0421
Epoch 15/30
186/186 [==============================] - 405s 2s/step - loss: 0.0424
Epoch 16/30
186/186 [==============================] - 404s 2s/step - loss: 0.0424
Epoch 17/30
186/186 

In [ ]:
stats_df = pd.DataFrame(data,columns=['year','train_df_length','test_df_length'])
stats_df.to_csv('Loss_values/CNN_stats_minmax.csv', header=False, index=False)

In [ ]:
import matplotlib.pylab as plt
data=[test_loss]
  
fig = plt.figure()  
# Creating axes instance 
ax = fig.add_axes([0, 0, 1, 1]) 
  
# Creating plot 
ax.boxplot(data)
years = [year for year in range(2008,2014)]
ax.set_xticklabels([year for year in range(2008,2014)]) 

# naming the y axis 
plt.ylabel('MSE Loss')
plt.title("Box plot for Test Loss")
textstr ='Test Loss for CNN : '+str(np.round(np.mean(test_loss),3))+' ('+str(np.round(np.std(test_loss),3))+')'
plt.gcf().text(0, -0.25, textstr, fontsize=14)
# show plot 
plt.savefig('Plots/block_plot_CNN_minmax.png',bbox_inches='tight')

In [ ]:
loss_data = []
for year_losses,year in zip(test_loss,years) :
    loss_data.append({'year':year,'value':year_losses})
loss_data_df = pd.DataFrame(loss_data,columns=['year','value'])
loss_data_df.to_csv('Loss_values/CNN_Loss_minmax.csv', header=False, index=False)